In [1]:
import tensorflow as tf
import numpy as np
import seq_nn
import pickle

D:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
fea_mat_path = 'D:/Data/MOT/save_fea_mat/MOT16-01.obj'
fea_label_path = 'D:/Data/MOT/save_fea_mat/MOT16-01_label.obj'
seq_model = 'D:/Data/UA-Detrac/cnn_MOT/model.ckpt'
fine_tune_model_path = 'D:/Data/UA-Detrac/fine_tune_model/model.ckpt'
max_length = 64
feature_size = 4+512
batch_size = 64
num_classes = 2

In [ ]:
batch_X_x = tf.placeholder(tf.float32, [None, 1, max_length, 1])
batch_X_y = tf.placeholder(tf.float32, [None, 1, max_length, 1])
batch_X_w = tf.placeholder(tf.float32, [None, 1, max_length, 1])
batch_X_h = tf.placeholder(tf.float32, [None, 1, max_length, 1])
batch_X_a = tf.placeholder(tf.float32, [None, feature_size-4, max_length, 1])
batch_mask_1 = tf.placeholder(tf.float32, [None, 1, max_length, 1])
batch_mask_2 = tf.placeholder(tf.float32, [None, feature_size-4, max_length, 1])
batch_Y = tf.placeholder(tf.int32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32)
    
y_conv = seq_nn.seq_nn(batch_X_x,batch_X_y,batch_X_w,batch_X_h,batch_X_a,batch_mask_1,
                        batch_mask_2,batch_Y,max_length,feature_size,keep_prob)
    
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=batch_Y, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(batch_Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()    
    
with tf.Session() as sess:
    
    saver.restore(sess, seq_model)
    print("Model restored.")

    batch_x = pickle.load(open(fea_mat_path,'rb'))
    batch_label = pickle.load(open(fea_label_path,'rb'))
    batch_size = batch_x.shape[0]
    
    remove_idx = []
    for n in range(batch_size):
        if np.sum(batch_x[n,0,:,1])==0:
            remove_idx.append(n)
    
    batch_x = np.delete(batch_x, np.array(remove_idx), axis=0)
    batch_label = np.delete(batch_label, np.array(remove_idx), axis=0)
    batch_size = batch_x.shape[0]
    batch_y = np.zeros((batch_size,2))
    for n in range(batch_size):
        if batch_label[n,2]==1:
            batch_y[n,0] = 1
        else:
            batch_y[n,1] = 1
     
    x = np.zeros((batch_size,1,max_length,1))
    y = np.zeros((batch_size,1,max_length,1))
    w = np.zeros((batch_size,1,max_length,1))
    h = np.zeros((batch_size,1,max_length,1))
    ap = np.zeros((batch_size,feature_size-4,max_length,1))
    mask_1 = np.zeros((batch_size,1,max_length,1))
    mask_2 = np.zeros((batch_size,feature_size-4,max_length,1))
    x[:,0,:,0] = batch_x[:,0,:,0]
    y[:,0,:,0] = batch_x[:,1,:,0]
    w[:,0,:,0] = batch_x[:,2,:,0]
    h[:,0,:,0] = batch_x[:,3,:,0]
    ap[:,:,:,0] = batch_x[:,4:,:,0]
    mask_1[:,0,:,0] = 1-batch_x[:,0,:,1]
    mask_2[:,:,:,0] = 1-batch_x[:,4:,:,1]

    train_accuracy = accuracy.eval(feed_dict={batch_X_x: x,
                                                batch_X_y: y,
                                                batch_X_w: w,
                                                batch_X_h: h,
                                                batch_X_a: ap,
                                                batch_mask_1: mask_1,
                                                batch_mask_2: mask_2,
                                                batch_Y: batch_y, 
                                                keep_prob: 1.0})
    print('step %d, training accuracy %g' % (train_accuracy))
    
    train_step.run(feed_dict={batch_X_x: x, 
                                      batch_X_y: y, 
                                      batch_X_w: w, 
                                      batch_X_h: h, 
                                      batch_X_a: ap, 
                                      batch_mask_1: mask_1, 
                                      batch_mask_2: mask_2, 
                                      batch_Y: batch_y, 
                                      keep_prob: 0.75})
    
    #save_path = saver.save(sess, fine_tune_model_path)
    #print("Model saved in path: %s" % save_path)
    
    train_accuracy = accuracy.eval(feed_dict={batch_X_x: x,
                                                batch_X_y: y,
                                                batch_X_w: w,
                                                batch_X_h: h,
                                                batch_X_a: ap,
                                                batch_mask_1: mask_1,
                                                batch_mask_2: mask_2,
                                                batch_Y: batch_y, 
                                                keep_prob: 1.0})
    print('step %d, training accuracy %g' % (train_accuracy))

INFO:tensorflow:Restoring parameters from D:/Data/UA-Detrac/cnn_appear_model_517_128_16600steps/model.ckpt
Model restored.
